## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-26-01-11-43 +0000,startribune,Takeaways: Defensive masterpiece lifts Vikings...,https://www.startribune.com/takeaways-defensiv...
1,2025-12-26-01-10-55 +0000,nyt,U.S. Strikes ISIS in Nigeria After Trump Warne...,https://www.nytimes.com/2025/12/25/us/politics...
2,2025-12-26-01-01-16 +0000,nypost,Grand jury declines to indict man in shooting ...,https://nypost.com/2025/12/25/us-news/grand-ju...
3,2025-12-26-01-00-00 +0000,wsj,Patients Who Searched in Vain for Mental-Healt...,https://www.wsj.com/us-news/law/health-insuran...
4,2025-12-26-01-00-00 +0000,wsj,The San Francisco Housing Wonk Trying to Becom...,https://www.wsj.com/politics/elections/who-is-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2390/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
71,christmas,40
11,trump,20
59,new,11
268,year,11
20,killed,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
149,2025-12-25-06-33-11 +0000,nypost,Trump-Kennedy Center Christmas Eve concert can...,https://nypost.com/2025/12/25/us-news/trump-ke...,88
108,2025-12-25-12-09-11 +0000,nyt,Kennedy Center’s Christmas Eve Jazz Show Cance...,https://www.nytimes.com/2025/12/24/us/kennedy-...,86
17,2025-12-25-23-24-19 +0000,nypost,Trump launches ‘powerful and deadly’ Christmas...,https://nypost.com/2025/12/25/us-news/trump-la...,86
70,2025-12-25-18-46-06 +0000,nypost,"Pope Leo XIV calls for peace in Middle East, U...",https://nypost.com/2025/12/25/world-news/pope-...,79
55,2025-12-25-19-28-21 +0000,nypost,New Yorkers shelled out more for Christmas gif...,https://nypost.com/2025/12/25/lifestyle/parent...,78


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
149,88,2025-12-25-06-33-11 +0000,nypost,Trump-Kennedy Center Christmas Eve concert can...,https://nypost.com/2025/12/25/us-news/trump-ke...
49,46,2025-12-25-19-55-37 +0000,nypost,3-year-old boy orphaned when wrong-way drunken...,https://nypost.com/2025/12/25/us-news/texas-bo...
70,39,2025-12-25-18-46-06 +0000,nypost,"Pope Leo XIV calls for peace in Middle East, U...",https://nypost.com/2025/12/25/world-news/pope-...
81,33,2025-12-25-17-20-17 +0000,nypost,Zelensky says Ukrainians just want Putin dead ...,https://nypost.com/2025/12/25/world-news/zelen...
115,30,2025-12-25-12-00-00 +0000,startribune,"Over the river and through the woods, ice skat...",https://www.startribune.com/over-the-river-and...
152,27,2025-12-25-05-12-45 +0000,latimes,San Diego man killed by falling tree in third ...,https://www.latimes.com/california/story/2025-...
130,26,2025-12-25-10-30-00 +0000,wsj,Big corporations are booming on AI and record ...,https://www.wsj.com/business/the-economic-divi...
39,25,2025-12-25-21-01-09 +0000,nyt,"No Power, No Heat, No Water: Odesa’s Days of H...",https://www.nytimes.com/2025/12/25/world/europ...
88,24,2025-12-25-16-37-49 +0000,cbc,King Charles says unity in diversity will help...,https://www.cbc.ca/news/world/king-charles-chr...
12,22,2025-12-25-23-59-19 +0000,latimes,Federal judge blocks ICE from arresting immigr...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
